# Segmenting and Clustering Neighborhoods in Toronto

In [90]:
# data from web

import pandas as pd
import numpy as np
import re

data_url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

data = pd.read_html(data_url)[0]

data_list = data.values.reshape(-1).tolist()

data_filter = []
PostalCode = []
Borough = []
Neighborhood =[]

for i in data_list:
    
    if 'Not assigned' not in i:
        i = i.replace(" / ",", ")
        data_filter.append(i)


for i in data_filter:
    char_posi = i.find('(')
    char_posi_end = i.find(')')
    
    i_Post = i[0:3]
    i_Boro = i[3:char_posi]
    i_Neigh = i[char_posi+1:char_posi_end]
    
    PostalCode.append(i_Post)
    Borough.append(i_Boro)
    Neighborhood.append(i_Neigh)
    
from pandas.core.frame import DataFrame

a = np.array(PostalCode).reshape(-1,1)
b = np.array(Borough).reshape(-1,1)
c = np.array(Neighborhood).reshape(-1,1)

sum = np.hstack((a,b,c))

sum[76][1] = 'Mississauga'
sum[76][2] ='Canada Post Gateway Processing Centre, Enclave of L4W'
sum[92][1] ='Downtown Toronto'
sum[92][2] ='Stn A PO Boxes25 The Esplanade, Enclave of M5E' 
sum[100][1] = 'East Toronto'
sum[100][2] = 'Business reply mail Processing Centre969 Eastern, Enclave of M4L'




data_df = DataFrame(sum)

data_df.columns=['PostalCode','Borough','Neighborhood']

data_df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [88]:
data_df.shape

(103, 3)